Este notebook es para ir haciendo pruebas con la augmentación de datos modificando sus parámetros para después usarlos en la red.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import shutil
import logging

import random
from random import shuffle

import imgaug as ia
import imgaug.augmenters as iaa



logger = tf.get_logger()
logger.setLevel(logging.ERROR)




In [2]:
def rescale_0_to_1(image):
    """
    Rescale image to [0, 1].
    :param image:
        Required. Image tensor.
    :return:
        Scaled image.
    """
    return tf.image.convert_image_dtype(image, tf.float64)

def rescale_min_1_to_1(image):
    """
    Rescale image to [-1, 1].
    :param image:
        Required. Image tensor.
    :return:
        Scaled image.
    """
    # Image must be casted to float32 first.
    image = tf.cast(image, tf.float64)
    # Rescale image from [0, 255] to [0, 2].
    image = tf.multiply(image, 1. / 127.5)
    # Rescale to [-1, 1].
    return tf.subtract(image, 1.0)


In [10]:
BRIGHTNESS_MIN_DELTA = 0.875
BRIGHTNESS_MAX_DELTA = 1.125

SATURATION_LOWER = 0.5
SATURATION_UPPER = 1.5

HUE_MIN_DELTA = 0.8
HUE_MAX_DELTA = 1.2

CONTRAST_LOWER = 0.5
CONTRAST_UPPER = 1.5

seq = iaa.Sequential([
    iaa.MultiplyBrightness((BRIGHTNESS_MIN_DELTA, BRIGHTNESS_MAX_DELTA)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5), 
    iaa.LinearContrast((CONTRAST_LOWER,CONTRAST_UPPER)),
    iaa.MultiplyHue((HUE_MIN_DELTA, HUE_MAX_DELTA)),
    iaa.MultiplySaturation((SATURATION_LOWER, SATURATION_UPPER)),
  
], random_order=True) # apply augmenters in random order


In [21]:
img_height=299
img_width=299
batch_size=16
train_dir = os.path.join( 'train8k_2cat')

image_gen_train = ImageDataGenerator(
      #preprocessing_function=rescale_min_1_to_1
)

train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(img_height,img_width),
                                                     class_mode='binary'
                                                     )

Found 4362 images belonging to 2 classes.


In [28]:
x,y= train_data_gen.next()

fig=plt.figure(figsize=(100,250))
columns = 2
rows = 8

for i in range(1, (columns*rows)+1, 2):
    image = x[i].astype(np.uint8)
    fig.add_subplot(rows, columns, i)
    plt.imshow(image)
    
    img1 = seq(image=image)
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img1)
    
    
plt.show()